# 📥 Import Libraries & Dataset

In [ ]:
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn==1.5.2
# %pip install xgboost
# %pip install catboost

## 📦 Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb # type: ignore
from sklearn.model_selection import RandomizedSearchCV, train_test_split #type: ignore
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report #type: ignore
from catboost import CatBoostClassifier #type: ignore

## 🗃️ Dataset

In [ ]:
# Import dataset
url = "https://drive.google.com/uc?id=1wnDUJCke0araT3A7SAk8ZLTIbV00R3ev"
df = pd.read_csv(url)

# 📰 Description

## 📍 Shape

In [ ]:
print("Jumlah baris: ", df.shape[0])
print("Jumlah kolom: ", df.shape[1])

Jumlah baris:  8000
Jumlah kolom:  21


## 📍 Structure

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     8000 non-null   object 
 1   VendorID               8000 non-null   float64
 2   lpep_pickup_datetime   8000 non-null   object 
 3   lpep_dropoff_datetime  8000 non-null   object 
 4   store_and_fwd_flag     8000 non-null   object 
 5   RatecodeID             8000 non-null   float64
 6   PULocationID           8000 non-null   int64  
 7   DOLocationID           8000 non-null   int64  
 8   passenger_count        8000 non-null   float64
 9   trip_distance          8000 non-null   float64
 10  fare_amount            8000 non-null   float64
 11  extra                  8000 non-null   float64
 12  mta_tax                8000 non-null   float64
 13  tip_amount             8000 non-null   float64
 14  tolls_amount           8000 non-null   float64
 15  ehai

## 📍 Columns

In [ ]:
df.columns

Index(['ID', 'VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'trip_type', 'congestion_surcharge', 'payment_type'],
      dtype='object')

Berikut ini adalah deskripsi kolom pada dataset:
- **VendorID**: Kode yang menunjukkan penyedia TPEP yang menyediakan catatan.
  - *1 = Creative Mobile Technologies, LLC*
  - *2 = VeriFone Inc*
- **lpep_pickup_datetime**: Tanggal dan waktu ketika meteran dinyalakan.
- **lpep_dropoff_datetime**: Tanggal dan waktu ketika meteran dimatikan.
- **store_and_fwd_flag**: Flag yang menunjukkan apakah catatan perjalanan disimpan dalam memori kendaraan sebelum dikirim ke vendor.
  - *Y = perjalanan yang disimpan dan diteruskan*
  - *N = perjalanan yang tidak disimpan dan diteruskan*
- **RatecodeID**: Kode tarif akhir yang berlaku pada akhir perjalanan.
  - *1 = Tarif standar*
  - *2 = JFK*
  - *3 = Newark*
  - *4 = Nassau atau Westchester*
  - *5 = Tarif negosiasi*
  - *6 = Perjalanan grup*
- **PULocationID**: Zona Taksi TLC di mana meteran dinyalakan.
- **DOLocationID**: Zona Taksi TLC di mana meteran dimatikan.
- **passenger_count**: Jumlah penumpang di kendaraan. Ini adalah nilai yang dimasukkan oleh sopir.
- **trip_distance**: Jarak perjalanan yang telah dilalui dalam mil yang dilaporkan oleh meteran.
- **fare_amount**: Biaya waktu dan jarak yang dihitung oleh meteran.
- **extra**: Biaya tambahan dan surcharge (misalnya, biaya tambahan $0,50 dan $1 untuk jam sibuk dan malam hari).
- **mta_tax**: Pajak MTA sebesar 0,50 dolar yang otomatis dikenakan berdasarkan tarif meteran yang digunakan.
- **tip_amount**: Jumlah tip field ini otomatis terisi untuk tip kartu kredit. Tip tunai tidak termasuk.
- **tolls_amount**: Jumlah total semua tol yang dibayar selama perjalanan.
- **improvement_surcharge**: Biaya perbaikan sebesar $0,30 yang dikenakan pada awal perjalanan. Biaya ini mulai diterapkan pada tahun 2015.
- **total_amount**: Jumlah total yang dibebankan kepada penumpang (tidak termasuk tip tunai).
- **payment_type**: Kode numerik yang menunjukkan bagaimana penumpang membayar perjalanan.
  - *1 = Kartu kredit*
  - *2 = Tunai*
- **trip_type**: Jenis perjalanan.
  - *1 = Inner city*
  - *2 = Outer city*
- **congestion_surcharge**: Jumlah total yang dikumpulkan untuk surcharge kemacetan NYC selama perjalanan.

## 📍 Overview

In [ ]:
df.head()

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T03315,2.0,2021-07-02 17:19:11,2021-07-02 17:40:02,N,1.0,152,142,1.0,4.30,...,1.0,0.5,5.00,0.00,NaN,0.3,27.55,1.0,2.75,1.0
1,T07720,2.0,2021-07-05 22:48:04,2021-07-05 23:12:02,N,1.0,93,107,2.0,13.52,...,0.5,0.5,12.15,6.55,NaN,0.3,60.75,1.0,2.75,1.0
2,T09695,2.0,2021-07-07 06:29:43,2021-07-07 06:38:42,N,1.0,74,75,1.0,1.55,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
3,T08802,2.0,2021-07-06 15:33:29,2021-07-06 15:43:25,N,1.0,226,129,1.0,1.22,...,0.0,0.5,0.00,0.00,NaN,0.3,8.80,1.0,0.00,2.0
4,T01413,2.0,2021-07-01 17:09:07,2021-07-01 18:03:12,N,1.0,55,86,1.0,14.23,...,1.0,0.5,0.00,2.45,NaN,0.3,49.75,1.0,0.00,1.0


In [ ]:
df.describe()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,0.0,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,1.833250,1.176375,98.700125,134.694875,1.270125,3.459532,14.995941,0.330781,0.475875,1.277159,0.351390,NaN,0.296887,18.237415,1.037875,0.599500,1.427625
std,0.372776,0.813541,64.792549,77.750038,0.917885,4.601964,14.543702,0.626111,0.113664,2.476832,1.503395,NaN,0.038558,16.126335,0.190906,1.136344,0.517972
min,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000,-25.000000,-1.000000,-0.500000,-1.140000,0.000000,NaN,-0.300000,-28.550000,1.000000,-2.750000,1.000000
25%,2.000000,1.000000,55.000000,74.000000,1.000000,1.030000,7.000000,0.000000,0.500000,0.000000,0.000000,NaN,0.300000,8.800000,1.000000,0.000000,1.000000
50%,2.000000,1.000000,75.000000,135.000000,1.000000,1.930000,10.200000,0.000000,0.500000,0.000000,0.000000,NaN,0.300000,13.300000,1.000000,0.000000,1.000000
75%,2.000000,1.000000,129.000000,212.000000,1.000000,3.790000,16.500000,0.500000,0.500000,2.040000,0.000000,NaN,0.300000,20.800000,1.000000,0.000000,2.000000
max,2.000000,5.000000,265.000000,265.000000,6.000000,37.600000,156.000000,4.500000,0.500000,47.880000,13.750000,NaN,0.300000,207.480000,2.000000,2.750000,4.000000


# 🛠️ Data Pre-Processing

## 📍 Duplicate Data

In [ ]:
# Cek duplikat data
print("Jumlah duplikasi data: ", df.duplicated().sum())

Jumlah duplikasi data:  0


## 📍 Missing Values

In [ ]:
# Threshold
MISSING_THRESHOLD = df.shape[0] * 0.5

missing_counts = df.isna().sum().sort_values(ascending=False)
present_counts = df.notna().sum()
missing_percentage = (missing_counts / df.shape[0] * 100).round(2)

missing_data_summary = pd.DataFrame({
    'Missing Values': missing_counts,
    'Present Values': present_counts,
    'Missing Percent': missing_percentage
})

missing_data_summary[missing_data_summary['Missing Values'] > MISSING_THRESHOLD]

,Missing Values,Present Values,Missing Percent
ehail_fee,8000,0,100.0


In [ ]:
# Drop ehail_fee
df = df.drop(columns=['ehail_fee'], axis=1)

In [ ]:
# Drop payment_type diluar 1, 2
df = df[df['payment_type'].isin([1, 2])]

# 🛠️ Feature Engineering

## 📍 Encoding

In [ ]:
# Encoding store_and_fwd_flag
label_encoder = LabelEncoder()

df["store_and_fwd_flag"] = label_encoder.fit_transform(df["store_and_fwd_flag"])

<ipython-input-751-4d1f3b02ad3b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["store_and_fwd_flag"] = label_encoder.fit_transform(df["store_and_fwd_flag"])


In [ ]:
# Convert datetime
df['pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

<ipython-input-752-4e39d24e1cfd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
<ipython-input-752-4e39d24e1cfd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])


In [ ]:
# Extract time-based features
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_day'] = df['pickup_datetime'].dt.dayofweek
df['pickup_month'] = df['pickup_datetime'].dt.month
df['dropoff_hour'] = df['dropoff_datetime'].dt.hour
df['dropoff_day'] = df['dropoff_datetime'].dt.dayofweek
df['dropoff_month'] = df['dropoff_datetime'].dt.month

<ipython-input-753-b652364b2b55>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pickup_hour'] = df['pickup_datetime'].dt.hour
<ipython-input-753-b652364b2b55>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pickup_day'] = df['pickup_datetime'].dt.dayofweek
<ipython-input-753-b652364b2b55>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
# Create trip_duration column
df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60

<ipython-input-754-9729dd1e0db9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trip_duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60


In [ ]:
def encode_cyclical(df, feature, max_val):
    df[f'{feature}_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
    df[f'{feature}_cos'] = np.cos(2 * np.pi * df[feature] / max_val)

In [ ]:
# Cyclinical encoding hour and month
encode_cyclical(df, 'pickup_hour', 24)
encode_cyclical(df, 'dropoff_hour', 24)
encode_cyclical(df, 'pickup_month', 12)
encode_cyclical(df, 'dropoff_month', 12)

<ipython-input-755-d4f3817cdc33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{feature}_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
<ipython-input-755-d4f3817cdc33>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{feature}_cos'] = np.cos(2 * np.pi * df[feature] / max_val)
<ipython-input-755-d4f3817cdc33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [ ]:
df.head()

,ID,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,...,dropoff_day,trip_duration,pickup_hour_sin,pickup_hour_cos,dropoff_hour_sin,dropoff_hour_cos,pickup_month_sin,pickup_month_cos,dropoff_month_sin,dropoff_month_cos
0,T03315,2.0,0,1.0,152,142,1.0,4.30,18.0,1.0,...,4,20.850000,-0.965926,-2.588190e-01,-0.965926,-2.588190e-01,-0.5,-0.866025,-0.5,-0.866025
1,T07720,2.0,0,1.0,93,107,2.0,13.52,38.0,0.5,...,0,23.966667,-0.500000,8.660254e-01,-0.258819,9.659258e-01,-0.5,-0.866025,-0.5,-0.866025
2,T09695,2.0,0,1.0,74,75,1.0,1.55,8.0,0.0,...,2,8.983333,1.000000,6.123234e-17,1.000000,6.123234e-17,-0.5,-0.866025,-0.5,-0.866025
3,T08802,2.0,0,1.0,226,129,1.0,1.22,8.0,0.0,...,1,9.933333,-0.707107,-7.071068e-01,-0.707107,-7.071068e-01,-0.5,-0.866025,-0.5,-0.866025
4,T01413,2.0,0,1.0,55,86,1.0,14.23,45.5,1.0,...,3,54.083333,-0.965926,-2.588190e-01,-1.000000,-1.836970e-16,-0.5,-0.866025,-0.5,-0.866025


In [ ]:
# One Hot Encoding Day
pickup_day_one_hot = pd.get_dummies(df['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
dropoff_day_one_hot = pd.get_dummies(df['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [ ]:
df = pd.concat([df, pickup_day_one_hot, dropoff_day_one_hot], axis=1)

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=['pickup_day', 'dropoff_day'], axis=1)

In [ ]:
df.head()

,ID,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,...,pickup_day_3,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
0,T03315,2.0,0,1.0,152,142,1.0,4.30,18.0,1.0,...,0,1,0,0,0,0,0,1,0,0
1,T07720,2.0,0,1.0,93,107,2.0,13.52,38.0,0.5,...,0,0,0,0,0,0,0,0,0,0
2,T09695,2.0,0,1.0,74,75,1.0,1.55,8.0,0.0,...,0,0,0,0,0,1,0,0,0,0
3,T08802,2.0,0,1.0,226,129,1.0,1.22,8.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,T01413,2.0,0,1.0,55,86,1.0,14.23,45.5,1.0,...,1,0,0,0,0,0,1,0,0,0


# 🛠️ Persiapan Dataset Testing (Kaggle)

In [ ]:
# Dataset Regresi
url = "https://drive.google.com/uc?id=192s2gBVRe7_5kTDkoIRM6Tls7SPNsnvf"
df_kaggle_regression = pd.read_csv(url)

# Dataset Klasifikasi
url = "https://drive.google.com/uc?id=192s2gBVRe7_5kTDkoIRM6Tls7SPNsnvf"
df_kaggle_classification = pd.read_csv(url)

In [ ]:
df_kaggle.head()

,Unnamed: 0,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,0,T00876,1.0,2021-07-01 13:41:08,2021-07-01 14:36:53,N,2.0,74,132,1.0,...,0.0,0.5,10.00,6.55,NaN,0.3,69.35,1.0,0.00,1.0
1,1,T01498,2.0,2021-07-01 18:33:34,2021-07-01 19:20:39,N,5.0,82,143,1.0,...,0.0,0.0,2.46,6.55,NaN,0.3,34.08,1.0,2.75,1.0
2,2,T08153,2.0,2021-07-06 09:37:59,2021-07-06 09:46:37,N,1.0,74,75,1.0,...,0.0,0.5,2.08,0.00,NaN,0.3,10.38,1.0,0.00,1.0
3,3,T03394,2.0,2021-07-02 18:24:34,2021-07-02 18:39:57,N,1.0,244,50,1.0,...,1.0,0.5,4.91,0.00,NaN,0.3,29.46,1.0,2.75,1.0
4,4,T09784,2.0,2021-07-07 08:30:59,2021-07-07 08:40:24,N,1.0,7,129,1.0,...,0.0,0.5,0.00,0.00,NaN,0.3,10.80,1.0,0.00,2.0


In [ ]:
df_kaggle = df_kaggle.drop(columns=['Unnamed: 0', 'ehail_fee'])

In [ ]:
df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     400 non-null    object 
 1   VendorID               400 non-null    float64
 2   lpep_pickup_datetime   400 non-null    object 
 3   lpep_dropoff_datetime  400 non-null    object 
 4   store_and_fwd_flag     400 non-null    object 
 5   RatecodeID             400 non-null    float64
 6   PULocationID           400 non-null    int64  
 7   DOLocationID           400 non-null    int64  
 8   passenger_count        400 non-null    float64
 9   fare_amount            400 non-null    float64
 10  extra                  400 non-null    float64
 11  mta_tax                400 non-null    float64
 12  tip_amount             400 non-null    float64
 13  tolls_amount           400 non-null    float64
 14  improvement_surcharge  400 non-null    float64
 15  total_

In [ ]:
processed_df_kaggle = df_kaggle.copy()

In [ ]:
processed_df_kaggle["store_and_fwd_flag"] = label_encoder.transform(processed_df_kaggle["store_and_fwd_flag"])

In [ ]:
processed_df_kaggle

,ID,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_type,congestion_surcharge,payment_type
0,T00876,1.0,2021-07-01 13:41:08,2021-07-01 14:36:53,0,2.0,74,132,1.0,52.00,0.0,0.5,10.00,6.55,0.3,69.35,1.0,0.00,1.0
1,T01498,2.0,2021-07-01 18:33:34,2021-07-01 19:20:39,0,5.0,82,143,1.0,22.02,0.0,0.0,2.46,6.55,0.3,34.08,1.0,2.75,1.0
2,T08153,2.0,2021-07-06 09:37:59,2021-07-06 09:46:37,0,1.0,74,75,1.0,7.50,0.0,0.5,2.08,0.00,0.3,10.38,1.0,0.00,1.0
3,T03394,2.0,2021-07-02 18:24:34,2021-07-02 18:39:57,0,1.0,244,50,1.0,20.00,1.0,0.5,4.91,0.00,0.3,29.46,1.0,2.75,1.0
4,T09784,2.0,2021-07-07 08:30:59,2021-07-07 08:40:24,0,1.0,7,129,1.0,10.00,0.0,0.5,0.00,0.00,0.3,10.80,1.0,0.00,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,T07978,2.0,2021-07-06 08:58:05,2021-07-06 09:24:58,0,1.0,75,136,1.0,25.00,0.0,0.5,3.00,0.00,0.3,28.80,1.0,0.00,1.0
396,T09337,2.0,2021-07-06 18:15:57,2021-07-06 18:16:50,0,1.0,193,193,1.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,1.0,0.00,1.0
397,T03284,1.0,2021-07-02 17:52:27,2021-07-02 17:59:55,0,1.0,95,95,1.0,6.50,1.0,0.5,1.70,0.00,0.3,10.00,1.0,0.00,1.0
398,T07319,2.0,2021-07-05 17:14:54,2021-07-05 17:25:24,0,1.0,95,95,1.0,9.50,0.0,0.5,0.00,0.00,0.3,10.30,1.0,0.00,2.0


In [ ]:
processed_df_kaggle['pickup_datetime'] = pd.to_datetime(processed_df_kaggle['lpep_pickup_datetime'])
processed_df_kaggle['dropoff_datetime'] = pd.to_datetime(processed_df_kaggle['lpep_dropoff_datetime'])

In [ ]:
processed_df_kaggle['pickup_hour'] = processed_df_kaggle['pickup_datetime'].dt.hour
processed_df_kaggle['pickup_day'] = processed_df_kaggle['pickup_datetime'].dt.dayofweek
processed_df_kaggle['pickup_month'] = processed_df_kaggle['pickup_datetime'].dt.month
processed_df_kaggle['dropoff_hour'] = processed_df_kaggle['dropoff_datetime'].dt.hour
processed_df_kaggle['dropoff_day'] = processed_df_kaggle['dropoff_datetime'].dt.dayofweek
processed_df_kaggle['dropoff_month'] = processed_df_kaggle['dropoff_datetime'].dt.month


In [ ]:
processed_df_kaggle['trip_duration'] = (processed_df_kaggle['dropoff_datetime'] - processed_df_kaggle['pickup_datetime']).dt.total_seconds() / 60

In [ ]:
def encode_test_cyclical(feature, max_val):
    processed_df_kaggle[f'{feature}_sin'] = np.sin(2 * np.pi * processed_df_kaggle[feature] / max_val)
    processed_df_kaggle[f'{feature}_cos'] = np.cos(2 * np.pi * processed_df_kaggle[feature] / max_val)

In [ ]:
encode_test_cyclical('pickup_hour', 24)
encode_test_cyclical('dropoff_hour', 24)
encode_test_cyclical('pickup_month', 12)
encode_test_cyclical('dropoff_month', 12)

In [ ]:
processed_df_kaggle = processed_df_kaggle.drop(columns=['pickup_hour', 'pickup_month', 'dropoff_hour', 'dropoff_month', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'pickup_datetime', 'dropoff_datetime'], axis=1)

In [ ]:
test_pickup_day_one_hot = pd.get_dummies(processed_df_kaggle['pickup_day'], prefix='pickup_day', drop_first=True).astype(int)
test_dropoff_day_one_hot = pd.get_dummies(processed_df_kaggle['dropoff_day'], prefix='dropoff_day', drop_first=True).astype(int)

In [ ]:
processed_df_kaggle = pd.concat([processed_df_kaggle, test_pickup_day_one_hot, test_dropoff_day_one_hot], axis=1)

In [ ]:
processed_df_kaggle = processed_df_kaggle.drop(columns=['pickup_day', 'dropoff_day'], axis=1)

In [ ]:
processed_df_kaggle.head()

,ID,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,fare_amount,extra,mta_tax,...,pickup_day_3,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
0,T00876,1.0,0,2.0,74,132,1.0,52.00,0.0,0.5,...,1,0,0,0,0,0,1,0,0,0
1,T01498,2.0,0,5.0,82,143,1.0,22.02,0.0,0.0,...,1,0,0,0,0,0,1,0,0,0
2,T08153,2.0,0,1.0,74,75,1.0,7.50,0.0,0.5,...,0,0,0,0,1,0,0,0,0,0
3,T03394,2.0,0,1.0,244,50,1.0,20.00,1.0,0.5,...,0,1,0,0,0,0,0,1,0,0
4,T09784,2.0,0,1.0,7,129,1.0,10.00,0.0,0.5,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
processed_df_kaggle = processed_df_kaggle.drop(columns="ID")

# 🛠️ Klasfikasi

## 📍 Training

In [ ]:
def evaluate_classifier_performance(prediction, y_test):
    # Informasi evaluasi secara compact
    print("Hasil Evaluasi berdasarkan classification report \n\n%s\n" % (classification_report(y_test, prediction,zero_division=0)))
    print()
    print("Confusion Matrix")
    print()
    y_actual = pd.Series(np.array(y_test), name = "actual")
    y_pred = pd.Series(np.array(prediction), name = "prediction")
    df_confusion = pd.crosstab(y_actual, y_pred)
    display(df_confusion)
    print()
    print()

    print("Butuh informasi lebih lengkap? silakan simak di bawah ini : ")
    print('Accuracy Average:', accuracy_score(y_test, prediction))
    print('F1 Macro Average:', f1_score(y_test, prediction, average='macro'))
    print('F1 Micro Average:', f1_score(y_test, prediction, average='micro'))
    print('Precision Macro Average:', precision_score(y_test, prediction, average='macro',zero_division=0))
    print('Precision Micro Average:', precision_score(y_test, prediction, average='micro',zero_division=0))
    print('Recall Macro Average:', recall_score(y_test, prediction, average='macro',zero_division=0))
    print('Recall Micro Average:', recall_score(y_test, prediction, average='micro',zero_division=0))
    print()

In [ ]:
df_classif = df.copy()

In [ ]:
# Drop kolom ID
df_classif = df_classif.drop(columns="ID")

In [ ]:
# Identify categorical features
# categorical_features = ['VendorID', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'trip_type']
categorical_features = ['store_and_fwd_flag']

In [ ]:
# Convert categorical columns to 'category' dtype
for col in categorical_features:
    df[col] = df[col].astype('category')

In [ ]:
# Pembagian fitur & target
X = df_classif.drop('payment_type', axis=1)
y = df_classif['payment_type']

In [ ]:
# Identify the indices of categorical features
cat_features_indices = [X.columns.get_loc(col) for col in categorical_features]

In [ ]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define hyperparameter grid for RandomizedSearchCV
param_grid = {
    'iterations': [500, 1000],
    'learning_rate': [0.03, 0.1],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [1, 3, 5],
    'random_strength': [0, 10, 100],
    'bagging_temperature': [0, 1, 5],
    'border_count': [32, 64, 128],
    'min_data_in_leaf': [1, 5, 10]
}

# Initialize CatBoost model
model = CatBoostClassifier(cat_features=cat_features_indices,
                           random_state=42, verbose=0)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions=param_grid,
                                   n_iter=50, scoring='f1', cv=3, verbose=1, n_jobs=-1)

In [ ]:
# Cheking Nan Values in X_train
print("Missing values in X_train: ", X_train.isna().sum().sum())

Missing values in X_train:  0


In [ ]:
X_train.head()

,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,pickup_day_3,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
5587,2.0,0,1.0,41,151,1.0,1.96,10.0,0.0,0.5,...,0,0,0,0,0,0,0,0,0,0
7911,2.0,0,1.0,247,42,6.0,1.17,8.0,1.0,0.5,...,0,1,0,0,0,0,0,1,0,0
3959,2.0,0,1.0,74,193,1.0,4.68,16.0,0.5,0.5,...,1,0,0,0,0,0,1,0,0,0
1908,2.0,0,1.0,41,41,1.0,0.63,4.5,0.0,0.5,...,0,0,0,0,1,0,0,0,0,0
6951,2.0,0,1.0,41,41,1.0,0.97,6.5,0.0,0.5,...,1,0,0,0,0,0,1,0,0,0


In [ ]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7922f75ecb20>,
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bagging_temperature': [0, 1, 5],
                                        'border_count': [32, 64, 128],
                                        'depth': [6, 8, 10],
                                        'iterations': [500, 1000],
                                        'l2_leaf_reg': [1, 3, 5],
                                        'learning_rate': [0.03, 0.1],
                                        'min_data_in_leaf': [1, 5, 10],
                                        'random_strength': [0, 10, 100]},
                   scoring='f1', verbose=1)

In [ ]:
best_params = random_search.best_params_
print(best_params)

best_model = random_search.best_estimator_
print(best_model)

{'random_strength': 10, 'min_data_in_leaf': 1, 'learning_rate': 0.03, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 6, 'border_count': 64, 'bagging_temperature': 0}


In [ ]:
y_pred = best_model.predict(X_test)

# Evaluasi performance
evaluate_classifier_performance(y_pred, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

         1.0       0.98      0.89      0.93       891
         2.0       0.87      0.97      0.92       696

    accuracy                           0.92      1587
   macro avg       0.92      0.93      0.92      1587
weighted avg       0.93      0.92      0.92      1587



Confusion Matrix



prediction,1.0,2.0
actual,,
1.0,791,100
2.0,20,676




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.9243856332703214
F1 Macro Average: 0.9239864864864864
F1 Micro Average: 0.9243856332703214
Precision Macro Average: 0.9232365540823979
Precision Micro Average: 0.9243856332703214
Recall Macro Average: 0.9295154611246566
Recall Micro Average: 0.9243856332703214



## 📍 Kaggle

In [ ]:
df_kaggle_classification = processed_df_kaggle.copy()

In [ ]:
y_result = best_model.predict(df_kaggle_classification)

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 6 should be feature with name trip_distance (found fare_amount).

In [ ]:
csv_result = pd.DataFrame({
    "ID": df_kaggle["ID"],
    "payment_type": y_result
})

In [ ]:
csv_result

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

utc_plus_7 = ZoneInfo("Asia/Bangkok")
current_iso_timestamp = datetime.now(utc_plus_7).isoformat()

filename = f"submission_kasdead_{current_iso_timestamp}.csv"
csv_result.to_csv(filename, index=False)